In [72]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from translate import Translator

In [73]:
coursera=  pd.read_csv("data/Coursera.csv", encoding="utf-8")
edx= pd.read_csv("data/EdX.csv", encoding="utf-8")
udacity= pd.read_csv("data/Udacity.csv", encoding="utf-8")

coursera['University'] = coursera['University'].apply(lambda x: 'É' + x[1:] if x.startswith('�') else 'é' + x[1:] if '�' in x else x)
coursera['University'] = coursera['University'].replace('École Polytechnique F�d�rale de Lausanne', 'École Polytechnique Fédérale de Lausanne')
# Add a new column named "Website" with the value "Coursera" in each row
coursera.insert(0, "Website", "Coursera")
edx.insert(0, "Website", "EdX")
udacity.insert(0, "Website", "Udacity")

coursera.head(5)

,Website,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Coursera,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Coursera,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Coursera,Silicon Thin Film Solar Cells,École Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Coursera,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Coursera,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [74]:
# Rename the "Course Type" column to "Type of Course"
coursera = coursera.rename(columns={"Course URL": "Link"})
coursera = coursera.rename(columns={"University": "Institution"})
coursera = coursera.rename(columns={"Course Name": "Name"})
coursera.head(5)

,Website,Name,Institution,Difficulty Level,Course Rating,Link,Course Description,Skills
0,Coursera,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Coursera,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Coursera,Silicon Thin Film Solar Cells,École Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Coursera,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Coursera,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [75]:
coursera.shape

(3522, 8)

In [76]:
edx = edx.rename(columns={"University": "Institution"})
edx.head(5)

,Website,Name,Institution,Difficulty Level,Link,About,Course Description
0,EdX,How to Learn Online,edX,Beginner,https://www.edx.org/course/how-to-learn-online,Learn essential strategies for successful onli...,"Designed for those who are new to elearning, t..."
1,EdX,Programming for Everybody (Getting Started wit...,The University of Michigan,Beginner,https://www.edx.org/course/programming-for-eve...,"This course is a ""no prerequisite"" introductio...",This course aims to teach everyone the basics ...
2,EdX,CS50's Introduction to Computer Science,Harvard University,Beginner,https://www.edx.org/course/cs50s-introduction-...,An introduction to the intellectual enterprise...,"This is CS50x , Harvard University's introduct..."
3,EdX,The Analytics Edge,Massachusetts Institute of Technology,Intermediate,https://www.edx.org/course/the-analytics-edge,"Through inspiring examples and stories, discov...","In the last decade, the amount of data availab..."
4,EdX,Marketing Analytics: Marketing Measurement Str...,"University of California, Berkeley",Beginner,https://www.edx.org/course/marketing-analytics...,This course is part of a MicroMasters® Program,Begin your journey in a new career in marketin...


In [77]:
edx.shape

(720, 7)

In [78]:
udacity = udacity.rename(columns={"School": "Institution"})
udacity = udacity.rename(columns={"Rating": "Course Rating"})
udacity.head(5)

,Website,Name,Institution,Difficulty Level,Course Rating,Link,About
0,Udacity,Data Engineer,School of Data Science,Intermediate,4.6,https://www.udacity.com//course/data-engineer-...,Data Engineering is the foundation for the new...
1,Udacity,Data Scientist,School of Data Science,Advanced,4.7,https://www.udacity.com//course/data-scientist...,"Build effective machine learning models, run d..."
2,Udacity,Data Analyst,School of Data Science,Intermediate,4.6,https://www.udacity.com//course/data-analyst-n...,"Use Python, SQL, and statistics to uncover ins..."
3,Udacity,C++,School of Autonomous Systems,Intermediate,4.6,https://www.udacity.com//course/c-plus-plus-na...,Get hands-on experience by building five real-...
4,Udacity,Product Manager,School of Product Management,Beginner,4.7,https://www.udacity.com//course/product-manage...,Envision and execute the development of indust...


In [79]:
edx.shape

(720, 7)

In [80]:
coursera.duplicated().sum()

98

In [81]:
coursera.drop_duplicates(inplace=True)

In [82]:
coursera.duplicated().sum()

0

In [83]:
udacity.duplicated().sum()

0

In [84]:
edx.duplicated().sum()

1

In [85]:
edx.drop_duplicates(inplace=True)

In [86]:
edx.duplicated().sum()

0

In [87]:
# Concatenate the datasets vertically
merged_data = pd.concat([coursera, edx, udacity], ignore_index=True)
merged_data.head(5000)


,Website,Name,Institution,Difficulty Level,Course Rating,Link,Course Description,Skills,About
0,Coursera,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...,NaN
1,Coursera,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...,NaN
2,Coursera,Silicon Thin Film Solar Cells,École Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...,NaN
3,Coursera,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...,NaN
4,Coursera,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...,NaN
...,...,...,...,...,...,...,...,...,...
4401,Udacity,Front-End Interview Prep,Career Advancement,Intermediate,None,https://www.udacity.com//course/front-end-inte...,NaN,NaN,Answer front-end technical and behavioral inte...
4402,Udacity,Full-Stack Interview Prep,Career Advancement,Intermediate,None,https://www.udacity.com//course/full-stack-int...,NaN,NaN,Answer common full stack and web security inte...
4403,Udacity,Data Structures & Algorithms in Swift,Career Advancement,Intermediate,None,https://www.udacity.com//course/data-structure...,NaN,NaN,Review and practice the skills technical inter...
4404,Udacity,iOS Interview Prep,Career Advancement,Intermediate,None,https://www.udacity.com//course/ios-interview-...,NaN,NaN,Answer iOS and mobile development interview qu...


In [88]:
merged_data.duplicated().sum()

0